# 大脑功能探索

这章来讲一下如何探索大脑的功能，大脑有很多区域，每个区域其实都有其特定的功能，人们做任何一件事都会使用大脑，例如，一个人动了一下手指头，仅仅是动了手指头这么简单的事情，大脑也是参与到其中运作了的。而要探索人动手指头和大脑哪片区域有关系则并不是一件容易的事情，下面将介绍如何探索大脑功能。 <br>

目前人类对大脑的最好的观测手段是用的核磁共振成像。这是个无需解剖大脑，只需要对着人脑扫描一下就可以得到大脑内部影像的方法。具体什么原理涉及到很多物理知识，比较复杂不说了。反正总之把人脑用这个核磁共振的机器扫一下就可以得到大脑图像。核磁共振成像分很多种，简单来说分两种，一种是结构图像（T1），还有一种是功能图像（T2）。所谓结构图像就是可以很清晰的显示出大脑各个部分的结构的图像，图像空间分辨率很高。功能图像空间分辨率不高，看着不太清楚，但是时间分辨率很高。**要想探索大脑功能就得使用fMRI，也就是功能核磁共振成像。**<br>

一个fMRI图像用张量表示大概是（D，H，W，T）这样的形式，前三个就是长宽高，最后T是时间。fMRI图像和我们平常接触到的RGB图像是不一样的，RGB图像是（w，h，3）只有长宽和3个颜色通道。fMRI图像则是个3D的volume，也就是体数据。RGB里面的点叫像素，范围是0到255。volume里面的点，准确说小正方体叫体素，范围？不清楚，我见过大到十万以上的。fMRI的体素代表了大脑活动信号的强度，数值越大表示大脑越活跃（个人理解）。这就是我们要探索的功能规律，要在活跃跳动的fMRI信号中，找出大脑在干什么。<br>

关于fMRI体素信号具体专业的说法是这些信号代表**血氧依赖水平(BOLD 对比度)**，测量血红蛋白与脱氧血红蛋白的比例，测量该组织的新陈代谢速度，间接反映神经元活动。BOLD的原理是：有氧血红蛋白是反磁性（Diamagnetic）的，脱氧血红蛋白是顺磁性（Paramagnetic）的。这两种不同状态下的蛋白一起作用产生了局部磁场。Bold fMRI（T2）正是利用了血红蛋白在这两种状态下不同的磁性性质，顺磁性的脱氧血红蛋白可以增强MR的原磁场。当它的含量下降时，fMRI信号也会跟着上升。静息态时，大脑的血流量和对应的T2信号值都是正常的。当神经元兴奋时，血流量增加，脱氧血红蛋白含量下降，于是T2信号随之上升。我们可以通过这个变化推断出神经元的活动。<br>

神经元兴奋时，对应的MR信号变化曲线被称为**血液动力响应函数(HRF,Hemodynamic Response Function)**。神经元一兴奋，就需要代谢掉一定的营养物质和氧气。氧气从血液中释放出来时，携氧血红蛋白迅速变成了顺磁性的脱氧血红蛋白，局部磁场发生变化，MR信号因此也有了一个快速的下降。这个过程叫做 **“启动下降”（Initial Dip）**<br>

启动下降过后，为了补充氧气的消耗，会有大量含携氧血红蛋白的新鲜血液流过来，还原态血红蛋白浓度下降。于是BOLD信号又会升高，HRF曲线逐渐增长，并在神经元兴奋后大约4到6秒达到顶峰(Peak)。在到达顶峰后，BOLD信号会大幅度的下降，甚至下降到基线以下（Baseline）。过度下降的原因是血流流出量逐渐减少，血流含量升高。<br>
![hrf](./img/HRF.png)
这个图片的意思是如果在0时刻发生了某些活动，BOLD信号就会发生这样的变化：首先上升，接着达到峰值，然后衰退并进入一段时间低谷，低于baseline十几秒。这是一个HRF的经验曲线。首先可以看到一个启动谷，曲线紧接着上升并在在兴奋后的5到6秒达到峰值，最后下降到基线之下，大约在25秒后恢复到基线水平。<br>

至于如何找出大脑的功能激活区域，通常的做法是事先设计好一个任务，让人去做这个事情，然后用fMRI检测你大脑的活动。这样在做任务的时候，大脑那个部位信号强度很大，突然变高，是不是就可以说明大脑这部分是负责你现在做的事情的工作的了？当然探索功能没有这么简单，因为即便你啥都没做，大脑每时每刻也是在活动的。对静息态模式下的大脑功能激活也是一个重要课题，但是这里暂时先不谈。<br>

刚才讲了脑部位信号变强表示脑在活动，这表示这个地方有可能和你目前做的任务相关。其实原理是这个原理，但是实际上还不够，因为大脑太复杂了，这里面噪音太多。这就需要选择统计模型，来处理他。过去一般是用的GLM也就是一般线性模型，或者字典学习，或者是ICA独立成分分析。<br>
存在的挑战：
- 数据不多，但是单个数据体量很大
- 信号的相关性很弱
- 数据有复杂的时序和空间噪音

## GLM
例如GLM是用的线性回归，$y=ax+b$，a就是系数，x就是任务设计，y就是fMRI的体素点，然后对回归系数a做个显著性检验，剩下通过了假设检验的a的系数就是激活的体素点。就是激活的功能区域，表示人做这个任务，大脑这块在工作。

## SDL稀疏字典学习
$Y_{T\times N}$其中Y表示大脑的fMRI信号矩阵，T表示T个时间步，N表示每个时间步的fMRI信号有N维。$D_{T\times K}$其中D表示字典矩阵，T表示T个时间步，K表示字典维度。$X_{K\times N}$其中X表示查询（稀疏）矩阵。其中Y是已知的fMRI信号，现在需要用D和X来近似学习对象Y。即
$$min||Y_{T\times N}-D_{T\times K}X_{K\times N}||^2$$
如果要获得稀疏解则需要添加L1正则项，即
$$min||Y_{T\times N}-D_{T\times K}X_{K\times N}||^2 + \lambda||X||_{L1}$$

## ICA独立成分分析
独立成分分析是想解决盲源分离的问题，在fMRI信号中可以理解为，原始的fMRI信号有多个源信号组成，他们相互叠加在一起是一个线性组合，形成了最终的fMRI信号。
$$X_{T\times N} = A_{T\times K}S_{K\times N}$$
其中X是混合信号，S是分离出来的源信号，A是一个未知的混合矩阵（mixing matrix），用来组合叠加信号S。K是人为设定的参数，表示有多少个源信号。



- 一般而言，GLM用于任务态的fMRI数据，ICA用于静息态的fMRI数据，SDL既可用于任务态也可以用于静息态。